In [4]:
import time
import torch
from vmas import make_env
from vmas.simulator.core import Agent
from vmas.simulator.scenario import BaseScenario
from typing import Union
from moviepy.editor import ImageSequenceClip
from IPython.display import HTML, display as ipython_display

class VMASEnvRunner:
    def __init__(
        self,
        render: bool,
        num_envs: int,
        num_episodes: int,
        max_steps_per_episode: int,
        device: str,
        scenario: Union[str, BaseScenario],
        continuous_actions: bool,
        random_action: bool,
        **kwargs
    ):
        self.render = render
        self.num_envs = num_envs
        self.num_episodes = num_episodes
        self.max_steps_per_episode = max_steps_per_episode
        self.device = device
        self.scenario = scenario
        self.continuous_actions = continuous_actions
        self.random_action = random_action
        self.kwargs = kwargs
        self.frame_list = []  # Initialize frame list

    def _get_deterministic_action(self, agent: Agent, env):
        if self.continuous_actions:
            print(f"agent action size: {agent.action_size}")
            print(f"environment batch dim: {env.batch_dim}")
            # action = -agent.action.u_range_tensor.expand(env.batch_dim, agent.action_size)
            action = -agent.action.u_range_tensor.expand(env.batch_dim, agent.action_size)
        else:
            action = torch.tensor([1], device=env.device, dtype=torch.long).unsqueeze(-1).expand(env.batch_dim, 1)
        return action.clone()

    def generate_gif(self, scenario_name):
        fps = 30
        clip = ImageSequenceClip(self.frame_list, fps=fps)
        clip.write_gif(f'{scenario_name}.gif', fps=fps)

        # Return the HTML tag to display the GIF
        return HTML(f'<img src="{scenario_name}.gif">')

    def run_vmas_env(self):
        scenario_name = self.scenario if isinstance(self.scenario, str) else self.scenario.__class__.__name__

        env = make_env(
            scenario=self.scenario,
            num_envs=self.num_envs,
            device=self.device,
            continuous_actions=self.continuous_actions,
            seed=0,
            **self.kwargs
        )

        init_time = time.time()
        total_steps = 0
        episode = 0

        for e in range(self.num_episodes):  # Loop over episodes
            episode += 1
            print(f"Episode {episode}")
            obs = env.reset()  # Reset environment at the start of each episode
            done = [False] * self.num_envs
            step = 0

            while not all(done) and step < self.max_steps_per_episode:  # Loop over steps within an episode
                step += 1
                total_steps += 1
                print(f"Step {step} of Episode {episode}")

                actions = []
                for i, agent in enumerate(env.agents):
                    if not self.random_action:
                        action = self._get_deterministic_action(agent, env)
                    else:
                        action = env.get_random_action(agent)

                    actions.append(action)
                    print(f"action agent {i}: {action} of {env.action_space}")
                
                obs, rews, dones, info = env.step(actions)
                done = [done or d for done, d in zip(done, dones)]

                if self.render:
                    frame = env.render(
                        mode="rgb_array",
                        agent_index_focus=None,
                    )
                    self.frame_list.append(frame)  # Append frame to the list

        total_time = time.time() - init_time
        print(
            f"It took: {total_time}s for {total_steps} steps across {self.num_episodes} episodes of {self.num_envs} parallel environments on device {self.device} "
            f"for {scenario_name} scenario."
        )

if __name__ == "__main__":

    scenario_name = "navigation_comm"

    env_runner = VMASEnvRunner(
        render=True,
        num_envs=1,
        num_episodes=1,
        max_steps_per_episode=10,
        device="cuda",
        scenario=scenario_name,
        continuous_actions=False,
        random_action=True,
        # Environment specific variables
        n_agents=2,
    )
    # Run the VMAS environment
    env_runner.run_vmas_env()

    # Generate and display the GIF
    ipython_display(env_runner.generate_gif(scenario_name))


Episode 1
Step 1 of Episode 1
action agent 0: tensor([5], device='cuda:0') of Tuple(Discrete(9), Discrete(9))
action agent 1: tensor([5], device='cuda:0') of Tuple(Discrete(9), Discrete(9))
Step 2 of Episode 1
action agent 0: tensor([4], device='cuda:0') of Tuple(Discrete(9), Discrete(9))
action agent 1: tensor([7], device='cuda:0') of Tuple(Discrete(9), Discrete(9))
Step 3 of Episode 1
action agent 0: tensor([3], device='cuda:0') of Tuple(Discrete(9), Discrete(9))
action agent 1: tensor([4], device='cuda:0') of Tuple(Discrete(9), Discrete(9))
Step 4 of Episode 1
action agent 0: tensor([0], device='cuda:0') of Tuple(Discrete(9), Discrete(9))
action agent 1: tensor([1], device='cuda:0') of Tuple(Discrete(9), Discrete(9))
Step 5 of Episode 1
action agent 0: tensor([6], device='cuda:0') of Tuple(Discrete(9), Discrete(9))
action agent 1: tensor([8], device='cuda:0') of Tuple(Discrete(9), Discrete(9))
Step 6 of Episode 1
action agent 0: tensor([5], device='cuda:0') of Tuple(Discrete(9), Dis